In [ ]:
import os
import shutil
import glob

%cd /kaggle/working

if os.path.exists("benchmarking-generative-models-for-domain-adaptation"):
    shutil.rmtree("benchmarking-generative-models-for-domain-adaptation")

!git clone --recursive https://github.com/AntonioRosano/benchmarking-generative-models-for-domain-adaptation.git

repo_path = "/kaggle/working/benchmarking-generative-models-for-domain-adaptation/models/munit"
%cd {repo_path}

!pip install -q tensorboardX

In [ ]:
files_to_fix = ['train.py', 'test.py', 'utils.py', 'trainer.py', 'networks.py']

for fname in files_to_fix:
    if os.path.exists(fname):
        with open(fname, 'r') as f:
            content = f.read()
        
        new_content = content
        
        # Fix 1: .item()
        new_content = new_content.replace('.data[0]', '.item()')
        
        # Fix 2: yaml loader
        new_content = new_content.replace('yaml.load(f)', 'yaml.load(f, Loader=yaml.FullLoader)')
        new_content = new_content.replace('yaml.load(stream)', 'yaml.load(stream, Loader=yaml.FullLoader)')
        
        # Fix 3: Rimozione load_lua (Causa crash utils.py)
        new_content = new_content.replace('from torch.utils.serialization import load_lua', '# from torch.utils.serialization import load_lua')
        
        if content != new_content:
            with open(fname, 'w') as f:
                f.write(new_content)
            print(f" -> Patch applicata a {fname}")
        else:
            print(f" -> {fname} già OK.")

In [ ]:
REAL_SOURCE = "/kaggle/input/datasets/marcogionfriddo/ego-ch-obj-seg/EGO-CH-OBJ-SEG/EGO-CH-OBJ-SEG/real/train/frames"
SYN_SOURCE = "/kaggle/input/datasets/marcogionfriddo/ego-ch-obj-seg/EGO-CH-OBJ-SEG/EGO-CH-OBJ-SEG/synthetic/train/frames"

WORK_DIR = os.getcwd()
DEST_DATASET = os.path.join(WORK_DIR, "datasets", "ego_adaptation")

trainA = os.path.join(DEST_DATASET, "trainA")
trainB = os.path.join(DEST_DATASET, "trainB")
testA = os.path.join(DEST_DATASET, "testA")
testB = os.path.join(DEST_DATASET, "testB")

if os.path.exists(DEST_DATASET):
    print("Pulisco vecchia cartella dataset...")
    shutil.rmtree(DEST_DATASET)

os.makedirs(trainA, exist_ok=True)
os.makedirs(trainB, exist_ok=True)
os.makedirs(testA, exist_ok=True)
os.makedirs(testB, exist_ok=True)

print(f"Copia immagini REALI (trainA)...")
!cp "{REAL_SOURCE}"/* "{trainA}/"

print(f"Copia immagini SINTETICHE (trainB)...")
!cp "{SYN_SOURCE}"/* "{trainB}/"

# POPOLAMENTO TEST (Anti-Crash)
def populate_folder_safe(src, dst, count=10):
    files = glob.glob(os.path.join(src, "*.*"))
    num_to_copy = min(count, len(files))
    print(f"Creating Test Set: Copio {num_to_copy} img da {os.path.basename(src)} a {os.path.basename(dst)}...")
    for i in range(num_to_copy):
        shutil.copy(files[i], dst)

populate_folder_safe(trainA, testA, 10)
populate_folder_safe(trainB, testB, 10)

print(f"\n DATASET PRONTO.")
print(f"TrainA: {len(os.listdir(trainA))} - TrainB: {len(os.listdir(trainB))}")
print(f"TestA:  {len(os.listdir(testA))} - TestB:  {len(os.listdir(testB))}")

In [ ]:
config_file = "configs/ego_kaggle_train.yaml"

munit_config = f"""
# MUNIT Config EGO-CH Kaggle
# logger options
image_save_iter: 1000         # Salva anteprima ogni 1000 step
image_display_iter: 1000      # Aggiorna log visuale
display_size: 4               # Mostra 4 immagini
snapshot_save_iter: 5000      # Salva modello (.pt) ogni 5000 step
log_iter: 100                 # Log testuale

# optimization options
max_iter: 75000               # ~25 Epoche
batch_size: 4                 # Ottimale per T4
weight_decay: 0.0001
beta1: 0.5                    
beta2: 0.999
init: kaiming
lr: 0.0001
lr_policy: step
step_size: 40000              # Decay dopo 40k
gamma: 0.5
gan_w: 1
recon_x_w: 10
recon_s_w: 1
recon_c_w: 1
recon_x_cyc_w: 0
vgg_w: 0                      # VGG disattivato per sicurezza

# model options
gen:
  dim: 64
  mlp_dim: 256
  style_dim: 8
  activ: relu
  n_downsample: 2
  n_res: 4
  pad_type: reflect
dis:
  dim: 64
  norm: none
  activ: lrelu
  n_layer: 4
  gan_type: lsgan
  num_scales: 3
  pad_type: reflect

# data options
input_dim_a: 3
input_dim_b: 3
num_workers: 4
new_size: 256                 # Resize sicuro per memoria
crop_image_height: 256        
crop_image_width: 256         
data_root: {DEST_DATASET}
"""

os.makedirs("configs", exist_ok=True)
with open(config_file, "w") as f:
    f.write(munit_config)

print(f"Configurazione salvata in: {config_file}")

In [ ]:
OUTPUT_DIR = "outputs/ego_munit_kaggle"

print("Avvio Training MUNIT...")
!python train.py --config configs/ego_kaggle_train.yaml --output_path {OUTPUT_DIR}